In [68]:
import copy
import sys
import numpy

In [5]:
#FILE PARSER: CREATE LIST OF CONNECTIONS
def file_parser(filename):
    newfile = open(filename)

    eof = False
    connect_list = []

    while eof == False:
        entry = newfile.readline()
        entry = entry.rstrip("\n")
        if entry == "END OF INPUT":
            eof = True
        else:
            entry_fix = entry.split(" ")
            connect_list.append(entry_fix)
    return connect_list

In [6]:
def generate_map(city_list):
    unique_cities = {}
    index = 0
    city_map = []
    
    for group in city_list:
        start, end, distance = group
        city_group_start = [start, [[end,int(distance)]]]
        city_group_end = [end, [[start,int(distance)]]]
        if start not in list(unique_cities.keys()):
            unique_cities[start] = index
            index += 1
            city_map.append(city_group_start)
        else:
            city_map[unique_cities[start]][1].append([end, int(distance)])
            
        if end not in list(unique_cities.keys()):
            unique_cities[end] = index
            index += 1
            city_map.append(city_group_end)
        else:
            city_map[unique_cities[end]][1].append([start, int(distance)])
            
        
    return city_map, unique_cities

In [7]:
blub = file_parser('programming_assignment1_input1.txt')
city_map, city_list = generate_map(blub)

city_map_og = copy.deepcopy(city_map)
city_list_og = copy.deepcopy(city_list)

In [86]:
city_map[city_list["Bremen"]]
cost = 5

for city in city_map[city_list["Bremen"]][1]:
    if len(city) == 2:
        city.append(city[1])
    city[1] += cost

print(city_map[city_list["Bremen"]])

for reset in city_map_og[city_list_og["Bremen"]][1]:
    city_map[city_list["Bremen"]][1] = city_map_og[city_list_og["Bremen"]][1]

print(city_map_og[city_list_og["Bremen"]][1])

['Bremen', [['Hamburg', 211, 116], ['Hannover', 227, 132], ['Dortmund', 329, 234]]]
[['Hamburg', 211, 116], ['Hannover', 227, 132], ['Dortmund', 329, 234]]


In [35]:
start = 'Bremen'
end = 'Frankfurt'


[['Hamburg', 116], ['Hannover', 132], ['Dortmund', 234]]


In [65]:
def pathfinder(next_node, end, fringe, cost, path_tracker, path_chosen, city_map, city_list):
    if next_node == end:
        return path_tracker[path_chosen]
    else:
        paths = city_map[city_list[next_node]][1] #all the connecting cities from the current city

        if path_tracker == []:
            for city in paths:
                path_tracker.append(city)
            print("path_tracker initialized to: " + str(path_tracker))
        
        print("PATH TRACKER UPDATE: " + str(path_tracker))
        
        #adding cities to the fringe, along with an extra cost variable that won't change
        for city in paths:
            """
            if city[0] in (i[0] for i in fringe):
                copycity = fringe[fringe.index(city)] #finds the duplicate city in the fringe
                city[1] += cost
                if copycity[1] > city[1]:
                    fringe.remove(copycity)
                    fringe.append(city)
            else: #the city is not already in the fringe 
            """
            city[1] += cost
            fringe.append(city)

        print(fringe)
        fringe.sort(key=lambda x: x[1])

        if end in (i[0] for i in fringe):
            print("goal found in fringe")
            return pathfinder(end, end, fringe, cost, path_tracker, path_chosen, city_map, city_list) #COST WILL NEED TO BE ADJUSTED HERE!!!
        
        else:                   #we'll take the lowest path UNLESS the goal exists in the fringe already
            
            next_node = fringe[0][0]
            new_cost = fringe[0][1]
            path_added = [next_node, new_cost]
            
            if path_chosen < 0:
                for nodes in path_tracker:
                    if [next_node, new_cost] == nodes:
                        path_chosen = path_tracker.index(nodes)
            
            path_tracker[path_chosen].append(path_added)

            if new_cost < cost: #tells us we have a better path somewhere else
                if path_added in (i[0] for i in path_tracker):
                    path_chosen = path_tracker.index(recall) #change what array we need to check
                    
            fringe.pop(0)

            return pathfinder(next_node, end, fringe, new_cost, path_tracker, path_chosen, city_map, city_list)

In [27]:
visited = [] # List for visited nodes.
queue = [] 

def bfs(visited, graph, node, city_list): #function for BFS
    
    visited.append(node)
    queue.append(node)

    while queue:          # Creating loop to visit each node
        #print("popped!")
        m = queue.pop(0) 
        #print(m) 

        for neighbour in graph[city_list[m]][1]:
            #print(neighbour)
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour[0])
                #print("visited: " + str(visited))
                #print("queue: " + str(queue))

    return visited
# Driver Code
print("Following is the Breadth-First Search")
looplist = bfs(visited, city_map, 'Bremen', city_list)    # function calling

Following is the Breadth-First Search


In [34]:
pathfinder('Bremen', 'Frankfurt', [], 0, [], [], -1):

Bremen


In [49]:
if "Frankfurt" not in (i[0] for i in looplist):
    print("no possible route")
else:
    print("possible route!")

possible route!


In [14]:
city_map[city_list["Bremen"]][1]

[['Hamburg', 116], ['Hannover', 132], ['Dortmund', 234]]

In [40]:
#THE BIG KAHUNA TRY 2 ELECTRIC BOOGALOO
def find_route(filename, start, end):
    #open the file and properly initialize everything
    city_file = file_parser(filename)
    city_map, unique_cities = generate_map(city_file)
    
    city_map_copy = copy.deepcopy(city_map)
    unique_cities_copy = copy.deepcopy(unique_cities)

    #run bfs to see if city distance is even possible from starting city
    looplist = bfs(visited, city_map, start, city_list)
    
    #CASE I: input cities are the same
    if start == end:
        #do the right output
        print("distance: 0 km")
        print("route:")
        print(start + " to " + end + ", 0 km")
        return
    
    #CASE II: input cities are not in the map at all
    elif (start not in list(unique_cities.keys())) or (end not in list(unique_cities.keys())):
        #do the right output
        print("distance: infinity")
        print("route:")
        print("none")
        return
    
    #CASE III: given cities are in the map, but no possible route exists between them
    elif (end not in (i[0] for i in looplist)):
        print("distance: infinity")
        print("route:")
        print("none")
        return
    
    #CASE IV: both cities are in the map and a route between them is possible
    else: #we are now sure the cities are different and a path can be found
        

SyntaxError: invalid syntax (742690688.py, line 26)

In [66]:
tester = file_parser('small_map.txt')
small_map, small_list = generate_map(tester)

pathfinder('A', 'F', [], 0, [], -1, small_map, small_list)

path_tracker initialized to: [['B', 1], ['C', 3]]
PATH TRACKER UPDATE: [['B', 1], ['C', 3]]
[['B', 1], ['C', 3]]
PATH TRACKER UPDATE: [['B', 1, ['B', 1]], ['C', 3]]
[['C', 3], ['A', 2], ['C', 4], ['E', 11]]
PATH TRACKER UPDATE: [['B', 1, ['B', 1], ['A', 2]], ['C', 3]]
[['C', 5], ['C', 4], ['E', 11], ['B', 3, ['B', 1], ['A', 2]], ['C', 5]]
PATH TRACKER UPDATE: [['B', 3, ['B', 1], ['A', 2], ['B', 3]], ['C', 5]]
[['C', 7], ['C', 5], ['C', 5], ['E', 14], ['A', 5], ['C', 7], ['E', 14]]
PATH TRACKER UPDATE: [['B', 3, ['B', 1], ['A', 2], ['B', 3], ['C', 5]], ['C', 5]]
[['C', 5], ['A', 5], ['C', 7], ['C', 7], ['E', 14], ['E', 14], ['A', 8], ['B', 8], ['F', 6], ['D', 9]]
goal found in fringe


['B', 3, ['B', 1], ['A', 2], ['B', 3], ['C', 5]]

In [69]:
i = [[1, 'a'], [2, 'c'], [1, 'f'] [4,'e']]

np.where(i == 1)

<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\hmvos\AppData\Local\Temp\ipykernel_3972\187195754.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  i = [[1, 'a'], [2, 'c'], [1, 'f'] [4,'e']]
C:\Users\hmvos\AppData\Local\Temp\ipykernel_3972\187195754.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  i = [[1, 'a'], [2, 'c'], [1, 'f'] [4,'e']]
C:\Users\hmvos\AppData\Local\Temp\ipykernel_3972\187195754.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  i = [[1, 'a'], [2, 'c'], [1, 'f'] [4,'e']]


TypeError: list indices must be integers or slices, not tuple